### Explanation of the data


- Gender :
Represents the student's gender (male or female).
A categorical feature often used in demographic analysis.

- EthnicGroup:
Represents the student's ethnic background (e.g., group A, group B, group C).
Another categorical variable that might relate to performance patterns.

- ParentEduc:
Indicates the highest level of education attained by the student’s parent (e.g., bachelor's degree, some college, master's degree).
A categorical variable that could correlate with the student's academic performance.

- LunchType:
Refers to the type of lunch the student receives (standard or free/reduced).
A socio-economic indicator that may influence academic performance.

- TestPrep:
Denotes whether the student completed test preparation (none, completed).
A categorical variable relevant to their scores.

- ParentMaritalStatus:
The marital status of the student’s parents (married, single, divorced).
May provide socio-demographic context.

- PracticeSport:
Frequency of the student practicing sports (regularly, sometimes, never).
Could impact mental and physical well-being, influencing academic performance.

- IsFirstChild:
Indicates if the student is the first child in the family (yes or no).
A categorical variable that might provide insight into family dynamics affecting education.

- NrSiblings:
The number of siblings the student has.
A numerical variable, potentially related to resource allocation within the family.

- TransportMeans:
How the student travels to school (e.g., school_bus, car, walk).
A categorical variable that might correlate with punctuality or attendance.

- WklyStudyHours:
The student’s weekly study hours (< 5, 5 - 10, etc.).
A categorical variable closely tied to academic performance.

- MathScore:
The student’s score in Math.
A numerical target variable for predictive tasks (e.g., regression).

- ReadingScore:
The student’s score in Reading.
Another numerical variable for prediction or analysis.

- WritingScore:
The student’s score in Writing.
Also a numerical variable, likely to be analyzed alongside Math and Reading scores.


# 1. Import Libraries and Load Data

dataset from kaggle:
https://www.kaggle.com/datasets/desalegngeb/students-exam-scores

In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # Encodes categorical data (e.g., strings) into numerical format.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# mean_squared_error < Measures the average squared difference between the actual and predicted values.
# r2_score Measures < how well the regression model fits the data. Values range from 0 (poor fit) to 1 (perfect fit).
from sklearn.impute import SimpleImputer

# Load datasets
df = pd.read_csv('/content/Expanded_data_with_more_features.csv')

# Preview the expanded dataset
df.head()


,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75


In [51]:
df.columns

Index(['Unnamed: 0', 'Gender', 'EthnicGroup', 'ParentEduc', 'LunchType',
       'TestPrep', 'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild',
       'NrSiblings', 'TransportMeans', 'WklyStudyHours', 'MathScore',
       'ReadingScore', 'WritingScore'],
      dtype='object')

In [52]:
# Drop the 'Unnamed: 0' column, as it is an index or irrelevant
df = df.drop(columns=['Unnamed: 0'])

In [53]:
df.shape

(30641, 14)

In [54]:
df.isnull().sum()

,0
Gender,0
EthnicGroup,1840
ParentEduc,1845
LunchType,0
TestPrep,1830
ParentMaritalStatus,1190
PracticeSport,631
IsFirstChild,904
NrSiblings,1572
TransportMeans,3134


In [55]:
# Handle missing values:
# For numerical columns, use the mean or median to fill missing values
numerical_columns = ['NrSiblings', 'WklyStudyHours', 'MathScore', 'ReadingScore', 'WritingScore']
numerical_imputer = SimpleImputer(strategy='mean')  # Can also use 'median' if necessary

# Convert the 'WklyStudyHours' column to numeric values
def convert_study_hours(value):
    if isinstance(value, str):  # Only process strings
        if value == "< 5":
            return 4  # Treat "< 5" as 4 hours of study
        elif value == "> 10":
            return 12  # Treat "> 10" as 12 hours of study
        elif value == "10-May":
            return 10  # Treat "10-May" as 10 hours of study
        elif ' - ' in value:  # For ranges like '5 - 10'
            # Extract the two numbers and return the average
            lower, upper = value.split(' - ')
            return (float(lower) + float(upper)) / 2
    # For non-string values (numerical), return the value as is
    return value

# Apply conversion function to the 'WklyStudyHours' column
df['WklyStudyHours'] = df['WklyStudyHours'].apply(convert_study_hours)

# Now handle missing values for the rest of the numerical columns
df[numerical_columns] = numerical_imputer.fit_transform(df[numerical_columns])

# Display the updated DataFrame
df


,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,4.0,71.0,71.0,74.0
1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,7.5,69.0,90.0,88.0
2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,4.0,87.0,93.0,91.0
3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,7.5,45.0,56.0,42.0
4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,7.5,76.0,78.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,7.5,59.0,61.0,65.0
30637,male,group E,high school,standard,none,single,regularly,no,1.0,private,7.5,58.0,53.0,51.0
30638,female,NaN,high school,free/reduced,completed,married,sometimes,no,1.0,private,7.5,61.0,70.0,67.0
30639,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,7.5,82.0,90.0,93.0


# 2. Data Preprocessing

In [56]:
# For categorical columns, use the mode to fill missing values
categorical_columns = ['EthnicGroup', 'ParentEduc', 'TestPrep', 'ParentMaritalStatus',
                       'PracticeSport', 'IsFirstChild', 'TransportMeans']

categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_columns] = categorical_imputer.fit_transform(df[categorical_columns])

# Now, handle categorical columns by encoding them
label_encoder = LabelEncoder()

# Encode categorical variables into numeric labels
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])   # LabelEncoder is used to convert categorical labels into numeric labels.

# 3. Split the Data into Features and Target Variables

In [57]:
df.isnull().sum()

,0
Gender,0
EthnicGroup,0
ParentEduc,0
LunchType,0
TestPrep,0
ParentMaritalStatus,0
PracticeSport,0
IsFirstChild,0
NrSiblings,0
TransportMeans,0


In [45]:
# Define features and target variables
X = df.drop(['MathScore', 'ReadingScore', 'WritingScore'], axis=1)
y_math = df['MathScore']
y_reading = df['ReadingScore']
y_writing = df['WritingScore']


# 4. Train-Test Split

In [58]:
# Split data into training and testing sets
X_train, X_test, y_train_math, y_test_math = train_test_split(X, y_math, test_size=0.2, random_state=42)
X_train, X_test, y_train_reading, y_test_reading = train_test_split(X, y_reading, test_size=0.2, random_state=42)
X_train, X_test, y_train_writing, y_test_writing = train_test_split(X, y_writing, test_size=0.2, random_state=42)


# 5. Model Training and Prediction (Linear Regression)

In [64]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

# Drop columns with all NaN values from X_train and X_test
X_train = X_train.dropna(axis=1, how='all')
X_test = X_test.dropna(axis=1, how='all')

# Recalculate the numerical and categorical columns after cleaning
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Initialize the imputers for numerical and categorical columns
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Apply the imputers only to non-empty numerical columns
X_train[numerical_cols] = numerical_imputer.fit_transform(X_train[numerical_cols])

# Apply the imputers only if categorical columns exist
if len(categorical_cols) > 0:
    X_train[categorical_cols] = categorical_imputer.fit_transform(X_train[categorical_cols])

# Apply the imputers on the test set as well
X_test[numerical_cols] = numerical_imputer.transform(X_test[numerical_cols])

# Apply the categorical imputer only if categorical columns exist
if len(categorical_cols) > 0:
    X_test[categorical_cols] = categorical_imputer.transform(X_test[categorical_cols])

# Check for NaNs in X_train and X_test
print(f"Checking NaN values in X_train: {X_train.isnull().sum().any()}")
print(f"Checking NaN values in X_test: {X_test.isnull().sum().any()}")
print(f"Checking NaN values in y_train_math: {y_train_math.isnull().sum()}")

# Initialize Linear Regression models
model_math = LinearRegression()
model_reading = LinearRegression()
model_writing = LinearRegression()

# Train the models
model_math.fit(X_train, y_train_math)
model_reading.fit(X_train, y_train_reading)
model_writing.fit(X_train, y_train_writing)

# Make predictions
y_pred_math = model_math.predict(X_test)
y_pred_reading = model_reading.predict(X_test)
y_pred_writing = model_writing.predict(X_test)


Checking NaN values in X_train: False
Checking NaN values in X_test: False
Checking NaN values in y_train_math: 0


# 6. Evaluate the Model Performance

In [78]:
# Display results
print("Math Score Predictions:", y_pred_math)
print("Reading Score Predictions:", y_pred_reading)
print("Writing Score Predictions:", y_pred_writing)


Math Score Predictions: [61.64089336 74.20736603 65.79404392 ... 54.82149008 70.701425
 66.9425741 ]
Reading Score Predictions: [68.27122158 76.1458421  68.60227211 ... 60.12806367 70.82182789
 69.14288629]
Writing Score Predictions: [68.31573838 77.45774296 66.78300769 ... 56.91062789 69.60060641
 67.60179065]


In [72]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Example classifier

# Example training data for classification task
training_data = pd.DataFrame({
    'Gender': ['female', 'male', 'female', 'male', 'female'],
    'EthnicGroup': ['group A', 'group B', 'group C', 'group A', 'group B'],
    'ParentEduc': ['bachelor\'s degree', 'high school', 'some college', 'high school', 'master\'s degree'],
    'LunchType': ['standard', 'free/reduced', 'standard', 'free/reduced', 'standard'],
    'TestPrep': ['none', 'completed', 'none', 'completed', 'none'],
    'ParentMaritalStatus': ['married', 'single', 'married', 'single', 'married'],
    'PracticeSport': ['regularly', 'never', 'regularly', 'never', 'regularly'],
    'IsFirstChild': ['yes', 'no', 'yes', 'no', 'yes'],
    'NrSiblings': [1, 3, 2, 0, 1],
    'TransportMeans': ['school_bus', 'car', 'car', 'school_bus', 'school_bus'],
    'WklyStudyHours': ['< 5', '5-10', '> 10', '5-10', '< 5'],
    'Performance': ['pass', 'fail', 'pass', 'fail', 'pass']  # Target variable
})

# Save feature names
training_feature_names = [
    'Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep',
    'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild',
    'NrSiblings', 'TransportMeans', 'WklyStudyHours'
]

# Encode categorical variables
label_encoders = {}
categorical_cols = [
    'Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep',
    'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild', 'TransportMeans', 'WklyStudyHours'
]

for col in categorical_cols:
    label_encoders[col] = LabelEncoder().fit(training_data[col])
    training_data[col] = label_encoders[col].transform(training_data[col])

# Split data into features and target
X = training_data[training_feature_names]
y = training_data['Performance']

# Encode the target variable
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a model (replace RandomForestClassifier with your model)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

# Print the classification report
report = classification_report(y_test, y_pred, target_names=target_encoder.classes_)
print("\nClassification Report:\n", report)


Model Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

        fail       1.00      1.00      1.00         1
        pass       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

